# Phylotrackpy with asynchronous generations example

This notebook contains a very simple evolving system with asynchronous generations 
wherein organisms reproduce as they accumulate the requisite resources.
Each organism's genome is a binary string, and organism's accumulate resources as 
a function of the proportion of 1s in their genome. 
I.e., this is a one-max environment!  

The code in this notebook does not incorporate phylogeny tracking (see [TODO] for 
a version of this code that _does_ already incorporate phylogeny tracking).
Think of this notebook as an opportunity to play around with how you could integrate
phylogeny tracking with `phylotrackpy` into an existing system.
Throughout the code, we have left comments where phylogeny tracking code should be added.
Below, we link to some existing examples and the `phylotrackpy` documentation to 
for your reference to get tracking working. 
If you're working on this during the tutorial at ALife 2024, feel free to ask for our help! 
We're happy to walk you through anything!  

## Helpful reference material

- `phylotrackpy` documentation: <https://phylotrackpy.readthedocs.io/en/latest/introduction.html>
- `phylotrackpy` GitHub repository: <https://github.com/emilydolson/phylotrackpy>
- Example code from an ALife 2023 tutorial: <https://github.com/emilydolson/alife-phylogeny-tutorial/blob/main/perfect_tracking_final.ipynb> 
  - Scroll down to the `phylotrackpy` heading!

## Setup

First, install required Python packages for this example. (e.g., `phylotrackpy`)

In [ ]:
!python3 -m pip install -r requirements.txt

In [23]:
from phylotrackpy import systematics
import random

In [24]:
# Seed random number generator
random.seed(8)

In [28]:
class Organism:
    def __init__(
        self,
        num_genes:int = 10,
        randomize_genome:bool = False,
        init_resources:float = 0.0
    ):
        # Genomes are vectors of binary values
        self.genome = [bool(random.randint(0, 1)) if randomize_genome else False for _ in range(num_genes)]
        # Organisms have resources that determine whether they can reproduce
        self.resources = init_resources
        # Organisms keep track of their taxon id (useful for phylogeny tracking)
        self.taxon_id = None

    @classmethod
    def FromGenome(cls, genome:list):
        # Create new organism from a given genome.
        org = cls(
            num_genes = 0,
            randomize_genome = False,
            init_resources = 0.0
        )
        org.SetGenome(genome)
        return org

    def GetGenome(self):
        return self.genome

    def SetGenome(self, genome:list):
        self.genome = [gene for gene in genome]

    def GetResources(self):
        return self.resources

    def SetResources(self, amount:float):
        self.resources = amount

    def IncResources(self, amount:float=1.0):
        self.resources += amount

    def DecResources(self, amount:float=1.0):
        self.resources -= amount

    def GetTaxonID(self):
        return self.taxon_id

    def SetTaxonID(self, id):
        self.taxon_id = id

    def Mutate(self, per_site_mut_rate:float=0.01):
        num_muts = 0
        for gene_i in range(len(self.genome)):
            if (random.random() <= per_site_mut_rate):
                self.genome[gene_i] = not self.genome[gene_i]
                num_muts += 1
        return num_muts


max_world_size = 500
updates = 1000
repro_cost = 10
max_resource_intake = 5
per_site_mut_rate = 0.05
genome_length = 100
assert(max_world_size > 0)

# Track based on genomes
phylo_tracker = systematics.Systematics(
    lambda org: org.GetGenome(),
    store_pos = False
)
phylo_tracker.add_snapshot_fun(
    lambda tax: str(list(map(int,tax.get_info()))).replace(",", " "),
    "genome",
    "Taxon's genome"
)
phylo_tracker.set_update(0)

# Create population seeded with initial organism
world = [Organism(num_genes=genome_length, randomize_genome=False)]
world[0].taxon_id = phylo_tracker.add_org(world[0])

print("Num initial taxa: ", phylo_tracker.get_num_taxa())

for update in range(0, updates):
    print(f"---Update {update}---")
    print(f"  Population size={len(world)}")
    print(f"  Max fitness={max([sum(world[i].GetGenome()) for i in range(len(world))])}")
    phylo_tracker.set_update(update)
    # Get ID of individual to "run"
    pop_id = random.randrange(0, len(world)) if len(world) > 1 else 0
    cur_org = world[pop_id]
    # print(f" Pop id={pop_id}")
    # print(f"  resources={cur_org.GetResources()}")
    # print(f"  genome={cur_org.GetGenome()}")
    # Calculate resource gain
    fit = sum(cur_org.GetGenome())
    # resource_gain = max(random.randint(0, fit) / max_resource_intake, 0.1)
    resource_gain = max((fit / genome_length) * max_resource_intake, 0.1)
    cur_org.IncResources(resource_gain)
    # print(f"  fit={fit}")
    # print(f"  resources gained={resource_gain}")
    # print(f"  resources={cur_org.GetResources()}")
    # Check if organism has resources to reproduce
    if cur_org.GetResources() >= repro_cost:
        # print(f"  reproduce!")
        # Current organism pays cost of reproduction
        cur_org.DecResources(repro_cost)
        # Create offspring as copy of current organism
        offspring = Organism.FromGenome(cur_org.GetGenome())
        mut_count = offspring.Mutate(per_site_mut_rate)
        offspring.taxon_id = phylo_tracker.add_org(offspring, cur_org.taxon_id)
        # Place offspring into world
        if len(world) < max_world_size:
            # If world not full, just append.
            world.append(offspring)
        else:
            # Otherwise, random replacement.
            offspring_loc = random.randint(0, len(world)-1)
            # Mark organism to be removed from phylogeny for deletion
            phylo_tracker.remove_org(world[offspring_loc].taxon_id)
            # Offspring replaces organism at chosen location.
            world[offspring_loc] = offspring

phylo_tracker.snapshot("final_snapshot.csv")


Num initial taxa:  1
---Update 0---
  Population size=1
  Max fitness=0
---Update 1---
  Population size=1
  Max fitness=0
---Update 2---
  Population size=1
  Max fitness=0
---Update 3---
  Population size=1
  Max fitness=0
---Update 4---
  Population size=1
  Max fitness=0
---Update 5---
  Population size=1
  Max fitness=0
---Update 6---
  Population size=1
  Max fitness=0
---Update 7---
  Population size=1
  Max fitness=0
---Update 8---
  Population size=1
  Max fitness=0
---Update 9---
  Population size=1
  Max fitness=0
---Update 10---
  Population size=1
  Max fitness=0
---Update 11---
  Population size=1
  Max fitness=0
---Update 12---
  Population size=1
  Max fitness=0
---Update 13---
  Population size=1
  Max fitness=0
---Update 14---
  Population size=1
  Max fitness=0
---Update 15---
  Population size=1
  Max fitness=0
---Update 16---
  Population size=1
  Max fitness=0
---Update 17---
  Population size=1
  Max fitness=0
---Update 18---
  Population size=1
  Max fitness=0
-